In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
from tensorflow import keras

from keras import backend as K
from keras.callbacks import EarlyStopping
from keras import models
from keras import layers

import sklearn.metrics as metrics
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import tensorflow.keras.backend 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import SGD
from keras import Sequential

import math,time,random

In [ ]:
df=pd.read_csv('./rawdata/BPI2016_Clicks_Logged_In.csv', sep=';', encoding='latin-1', keep_default_na=False)



df['time'] = pd.to_datetime(df['TIMESTAMP'])
df['dates'] = df['time'].dt.date


In [ ]:

wrkmsg=pd.read_csv('./rawdata/BPI2016_Werkmap_Messages.csv', encoding='latin-1', sep=';', keep_default_na=False)
wm_columns_to_keep = ['EventDateTime','CustomerID']

wrkmsg=wrkmsg[[c for c in wrkmsg.columns if c in wm_columns_to_keep]]
wrkmsg.columns = ['CustomerID','ContactDate']

wrkmsg['ContactDate'] = pd.to_datetime(wrkmsg['ContactDate'])
wrkmsg['ContactTimeStart'] = [dt.datetime.time(d) for d in wrkmsg['ContactDate']] 
wrkmsg['ContactDate'] = [dt.datetime.date(d) for d in wrkmsg['ContactDate']] 
wrkmsg.head()
wrkmsg_customers=(df[df['CustomerID'].isin(wrkmsg['CustomerID'].unique())])
wrkmsg_customers.shape

In [ ]:
wrkmsg['dates']=wrkmsg['ContactDate']
wrkmsg_customers_we=wrkmsg_customers[wrkmsg_customers['page_load_error']==0]
# session_interaction = pd.merge(wrkmsg_customers_we, wrkmsg, how="left", on=["CustomerID", "dates"])
# session_interaction.shape

In [ ]:
wrkmsg_customers_we.shape

In [ ]:
df.head()

In [ ]:
print("Unique traces count", len(df.SessionID.unique()))

In [ ]:
print("Unique events count",len(df.PAGE_NAME.unique()))

In [ ]:
print("Total events count",len(df.PAGE_NAME))

In [ ]:
print(len(df.PAGE_NAME)/len(df.SessionID.unique()))

In [ ]:
wrkmp_custdates=(wrkmsg.groupby(['CustomerID','dates'])).agg(count=('ContactTimeStart', 'count')).reset_index()
session_wrk = pd.merge(wrkmsg_customers_we, wrkmp_custdates, how="left", on=["CustomerID", "dates"])
session_wrk.shape

In [ ]:
session_wrk.head()

#TO BE ADDED AS A LAYER?
cust_info = session_wrk.loc[:,['CustomerID','AgeCategory','Gender','Office_U','Office_W']].drop_duplicates()

In [ ]:
# session_wrk = session_wrk.drop(['SessionID','AgeCategory','Gender','Office_U','Office_W','page_load_error'], axis=1)
session_wrk = session_wrk.drop(['Office_U','Office_W','page_load_error'], axis=1)
session_wrk.loc[:,'count']=session_wrk.loc[:,'count'].fillna(0)

In [ ]:
session_wrk['Flag'] = np.where(session_wrk['count']>0, 1, 0)
cust_ids = session_wrk['CustomerID'].unique()

In [ ]:
session_wrk[session_wrk.page_action_detail!='']['SessionID']

In [ ]:
counts = session_wrk['PAGE_NAME'].value_counts()
# print (counts[:])
thres=session_wrk.shape[0]*0.005
session_wrk_fil=session_wrk[session_wrk['PAGE_NAME'].isin(counts[counts > thres].index)]
print(len(session_wrk_fil.PAGE_NAME.unique()))
counts1 = session_wrk_fil['PAGE_NAME'].value_counts()
print (counts1[:])

In [ ]:
(session_wrk[session_wrk.SessionID==44265671].groupby(['SessionID'])).agg(page_count=('PAGE_NAME', 'count'), flag=('Flag','max')).reset_index()
# ['page_action_detail']

In [ ]:
(session_wrk_fil[session_wrk_fil.SessionID==44265671].groupby(['SessionID'])).agg(page_count=('PAGE_NAME', 'count'), flag=('Flag','max')).reset_index()

In [ ]:
# session_wrk_fil_pc_s=(session_wrk_fil.groupby(['SessionID'])).agg(page_count=('PAGE_NAME', 'nunique')).reset_index()
# session_wrk_fil_pc=(session_wrk_fil_pc_s.groupby(['page_count'])).agg(num_sessions=('SessionID', 'nunique')).reset_index()


session_wrk_fil_pc_s=(session_wrk_fil.groupby(['SessionID'])).agg(page_count=('PAGE_NAME', 'count'), flag=('Flag','max')).reset_index()
session_wrk_fil_pc=(session_wrk_fil_pc_s.groupby(['page_count','flag'])).agg(num_sessions=('SessionID', 'count')).reset_index()
tot_sess = len(session_wrk_fil['SessionID'].unique())
session_wrk_fil_pc['perc_sess']=session_wrk_fil_pc['num_sessions']/tot_sess


In [ ]:
session_wrk_fil_pc_s[session_wrk_fil_pc_s.SessionID==44265671]

In [ ]:
session_wrk_fil_pc.head(20)

In [ ]:
lookback=7
# min_los = (lookback/2)
min_los = 3
sessions_to_keep = session_wrk_fil_pc_s[(session_wrk_fil_pc_s['page_count']>=min_los) & (session_wrk_fil_pc_s['page_count']<=lookback)]['SessionID']
# sessions_to_keep = session_wrk_fil_pc_s[(session_wrk_fil_pc_s['page_count']>=3 & (session_wrk_fil_pc_s['page_count']<=8))]['SessionID']
# sessions_to_keep = session_wrk_fil_pc_s[(session_wrk_fil_pc_s['page_count']==lookback)]['SessionID']

session_wrk_lkbk=session_wrk_fil[session_wrk_fil['SessionID'].isin(sessions_to_keep)]

In [ ]:
session_wrk_lkbk

In [ ]:
session_wrk_lkbk.shape
sessions_to_keep.shape
session_wrk_lkbk.head()

In [ ]:
session_wrk_lkbk['page_action_detail'].unique()

In [ ]:
session_wrk_lkbk[session_wrk_lkbk['SessionID']==44265671]

In [ ]:
session_wrk_fil.dtypes

In [ ]:
# session_wrk_fil.loc[:,'dates'] =  session_wrk_fil.loc[:,'dates'].astype(str)


encoder = LabelEncoder() #Label encoder may introduce order; One-HOt encodingg!!

session_wrk_fil.loc[:,'CustomerID'] =  encoder.fit_transform(session_wrk_fil.loc[:,'CustomerID'])
# session_wrk_fil.loc[:,'TIMESTAMP'] =  encoder.fit_transform(session_wrk_fil.loc[:,'TIMESTAMP'])
session_wrk_fil.loc[:,'PAGE_NAME'] =  encoder.fit_transform(session_wrk_fil.loc[:,'PAGE_NAME'])
session_wrk_fil.loc[:,'VHOST'] =  encoder.fit_transform(session_wrk_fil.loc[:,'VHOST'])
session_wrk_fil.loc[:,'URL_FILE'] =  encoder.fit_transform(session_wrk_fil.loc[:,'URL_FILE'])
session_wrk_fil.loc[:,'xps_info'] =  encoder.fit_transform(session_wrk_fil.loc[:,'xps_info'])
session_wrk_fil.loc[:,'dates'] =  encoder.fit_transform(session_wrk_fil.loc[:,'dates'])
session_wrk_fil.loc[:,'REF_URL_category'] =  encoder.fit_transform(session_wrk_fil.loc[:,'REF_URL_category'])
session_wrk_fil.loc[:,'page_action_detail_EN'] =  encoder.fit_transform(session_wrk_fil.loc[:,'page_action_detail_EN'])
session_wrk_fil.loc[:,'service_detail_EN'] =  encoder.fit_transform(session_wrk_fil.loc[:,'service_detail_EN'])
session_wrk_fil.loc[:,'tip_EN'] =  encoder.fit_transform(session_wrk_fil.loc[:,'tip_EN'])


data = session_wrk_fil.copy()

categorical_cols = [ 'PAGE_NAME'] 
cols_to_drop = ['IPID','tip','service_detail','page_action_detail','count','dates','xps_info', 'VHOST', 'URL_FILE' ,'page_action_detail_EN', 'tip_EN', 'REF_URL_category', 'service_detail_EN']

data = data.drop(cols_to_drop, axis=1)


data.sort_values(by=['CustomerID','TIMESTAMP'],ascending=False,inplace=True)

data.head(10)

In [ ]:
def add_timespent(df):
    return_df = pd.DataFrame(columns = df.columns)
#     return_df=session_interaction
    unique_sess =  df['SessionID'].unique()
    for sess in unique_sess:
        session_level = df.loc[df['SessionID']==sess]
        session_level = df.sort_values(by=['SessionID','TIMESTAMP'])
        session_level.loc[:,'timespent']=session_level.loc[:,'time'].diff().apply(lambda x: x/np.timedelta64(1, 's')).fillna(0).astype('int64')
        return_df=return_df.append(session_level,ignore_index=True)
#         print(return_df.head())
    return return_df

In [ ]:
data = session_wrk_lkbk.copy()

categorical_cols = [ 'PAGE_NAME', 'AgeCategory', 'Gender'] 
cols_to_drop = ['page_action_detail','IPID','tip','service_detail','page_action_detail_EN','count','dates','xps_info', 'VHOST', 'URL_FILE' , 'tip_EN', 'REF_URL_category', 'service_detail_EN']

data = data.drop(cols_to_drop, axis=1)

data.sort_values(by=['CustomerID','TIMESTAMP'],ascending=False,inplace=True)

data.loc[:,'CustomerID'] =  session_wrk_fil.loc[:,'CustomerID'].astype(str)

data.loc[:,'TIMESTAMP']=(data['time']).astype(int)/ 10**9


# data.loc[:,'TIMESTAMP']= data['TIMESTAMP'].transform( lambda x: x-x.min())

# scaler = MinMaxScaler(feature_range=(0, 1))
# data.loc[:,'TIMESTAMP'] =  scaler.fit_transform(data[['TIMESTAMP']])



encoder = LabelEncoder() #Label encoder may introduce order; One-HOt encodingg!!

# data.loc[:,'CustomerID'] =  encoder.fit_transform(data.loc[:,'CustomerID'])

data = data.drop(['CustomerID'], axis=1)

# data = add_timespent(data)

data = data.drop(['time'], axis=1)

encoded_data = pd.get_dummies(data, columns = categorical_cols) #TO TRY - binary encoding or other encodings

#Customer visit features - 

In [ ]:
session_wrk_lkbk['page_action_detail'].unique()

In [ ]:
print(len(session_wrk_lkbk.SessionID.unique()))

session_wrk_fil['TIMESTAMP'].hist()

In [ ]:
data['TIMESTAMP'].hist()

In [ ]:
encoded_data.head()

In [ ]:
def class_balancer(d):
    sess_K1 = d[d.Flag == 1]['SessionID']
    sess_K0 = d[d.Flag == 0]['SessionID'].unique()
    print(len(sess_K0))
    print(len(sess_K0))
    smsk = np.random.rand(len(sess_K0)) > 0.81
    sess2keep = sess_K0[smsk]
    print(len(sess2keep))
    fl = np.append(sess2keep,sess_K1)
#     sess2keep = sess2keep.append(sess_K1)
    
    return d[d.SessionID.isin(fl)]

In [ ]:
# 3 way split
cust_ids = encoded_data['SessionID'].unique()

msk = np.random.rand(len(cust_ids)) < 0.5
train_ids = cust_ids[msk]
val_test_ids = cust_ids[~msk]

train = encoded_data[encoded_data['SessionID'].isin(train_ids)]

msk = np.random.rand(len(val_test_ids)) < 0.5
validate_ids = val_test_ids[msk]
test_ids = val_test_ids[~msk]

validate = encoded_data[encoded_data['SessionID'].isin(validate_ids)]
test = encoded_data[encoded_data['SessionID'].isin(test_ids)]


# train = class_balancer(train)
# validate = class_balancer(validate)
# test = class_balancer(test)


train_target = train.loc[:,"Flag"]
validate_target = validate.loc[:,"Flag"]
test_target = test.loc[:,"Flag"]

print(len(train[train.Flag==1]['SessionID'].unique())/len(train['SessionID'].unique())) #Sessions with flag =1 
print(len(validate[validate.Flag==1]['SessionID'].unique())/len(validate['SessionID'].unique())) #Sessions with flag =1 
print(len(test[test.Flag==1]['SessionID'].unique())/len(test['SessionID'].unique())) #Sessions with flag =1 

# train_data = train.drop(['Flag','SessionID'], axis=1)
# validate_data = validate.drop(['Flag','SessionID'], axis=1)
# test_data = test.drop(['Flag','SessionID'], axis=1)
train_data = train.drop(['Flag'], axis=1)
validate_data = validate.drop(['Flag'], axis=1)
test_data = test.drop(['Flag'], axis=1)

In [ ]:
# subset sesss with KPI =1
# from remaining sample!!
# d = validate
validate_ids.sort()

In [ ]:
validate_ids

print(len(validate[validate.Flag==1]['SessionID'].unique())/len(validate['SessionID'].unique())) #Sessions with flag =1 


a=class_balancer(validate)
len(a.SessionID.unique())
print(len(a[a.Flag==1]['SessionID'].unique())/len(a['SessionID'].unique())) #Sessions with flag =1 


In [ ]:
def pivot_data(data):
#     data=data.drop('Flag', axis=1)
    data=data.sort_values('SessionID')
    ndata = data.copy()
    unique_sess = data['SessionID'].unique()
    print(len(unique_sess))
#     x = np.empty(data.shape[0])
    for id in unique_sess:
        sub_data = data.loc[data.SessionID == id,:]
        
        sub_data.loc[:,'TIMESTAMP'] = sub_data.loc[:,'TIMESTAMP'].transform( lambda x: x-x.min())
        ndata.loc[ndata.SessionID == id,:] = sub_data.copy()
        
    cols = ndata.columns[2:]
#     print(cols)
    print(ndata.columns)
    s1 = ndata[ndata.columns[:2]]
    s2 = ndata[cols].idxmax(axis=1)
    nd = pd.concat([s1,s2],axis=1,keys=None)
    print(nd)
    nd.rename(columns={ nd.columns[2]: "PAGE_NAME" }, inplace = True)
    session_level = nd.pivot_table(index='SessionID', columns='PAGE_NAME', values=['TIMESTAMP']).reset_index()
#         rdf=pd.concat(rdf,session_level)
    return session_level.fillna(0)


In [ ]:
# pivot_data(test_data.head(20))
# test.head(10)[test.columns[:3]]

In [ ]:
def reshape_data(data, lookback, nsamples):
    data=data.sort_values(['SessionID','TIMESTAMP'])
    unique_sess = data.SessionID.unique()
    rdf=[]
    j=len(data.columns)
    k=lookback
    x = np.empty((nsamples,j,k))
    i=0
    for id in unique_sess:
        sub_data = data[data.SessionID == id]
#         sub_data = sub_data.drop('SessionID', axis=1)
        sub_data = sub_data.drop(['SessionID','TIMESTAMP'], axis=1)
#         sub_data.loc[:,'TIMESTAMP']= sub_data['TIMESTAMP'].transform( lambda x: (x-x.min()/(x.max())))
#         sub_data.loc[:,'TIMESTAMP']= sub_data['TIMESTAMP'].transform( lambda x: x-x.min())
        temp2d = np.transpose(keras.preprocessing.sequence.pad_sequences(np.transpose(sub_data.values), maxlen=lookback, dtype='float32', value=0., padding='pre'))
        rdf.append(temp2d)
        i += 1
    return np.stack(rdf)

In [ ]:
train_nsamples = len(train_data.SessionID.unique())
test_nsamples = len(test_data.SessionID.unique())
validate_nsamples = len(validate_data.SessionID.unique())

In [ ]:
train_data.sort_values('SessionID')

In [ ]:
train_rsd = reshape_data(train_data, lookback, train_nsamples)
test_rsd = reshape_data(test_data, lookback, test_nsamples)
validate_rsd = reshape_data(validate_data, lookback, validate_nsamples)

In [ ]:
print(train_rsd.shape)
print(test_rsd.shape)
print(validate_rsd.shape)

train_rsd = np.take(train_rsd, dummy_idx_to_keep, axis = 0)
train_target = np.take(train_target, dummy_idx_to_keep, axis = 0)

In [ ]:
# len(to_keep)

In [ ]:
# to_keep = np.unique(to_keep)

In [ ]:
# to_keep

In [ ]:
# cases_to_keep[0]

In [ ]:
# (cases_to_keep[0]).extend(to_keep)

In [ ]:
# len(cases_to_keep)

In [ ]:
# cases_to_keep[1]

In [ ]:
def class_balancer(d):
    sess_K1 = d[d.Flag == 1]['SessionID']
    sess_K0 = d[d.Flag == 0]['SessionID'].unique()
    print(len(sess_K0))
    print(len(sess_K0))
    smsk = np.random.rand(len(sess_K0)) > 0.81
    sess2keep = sess_K0[smsk]
    print(len(sess2keep))
    fl = np.append(sess2keep,sess_K1)
#     sess2keep = sess2keep.append(sess_K1)
    
    return d[d.SessionID.isin(fl)]

import time
st = time.time()

print(validate_rsd.shape, test_rsd[0:10].shape)

a = torch.flatten(torch.tensor(test_rsd[0:10]), start_dim=1)  #simulates running traces
b = torch.flatten(torch.tensor(validate_rsd), start_dim=1) #dataset to find candidates from, also used for tuning the model 

print(a.shape, b.shape)


# print(torch.cdist(a, b).topk(100, largest=False)[1][0:10])

print( time.time() - st)

train_nsamples = int((train_data.shape[0])/lookback)
test_nsamples = int((test_data.shape[0])/lookback)
validate_nsamples = int((validate_data.shape[0])/lookback)

# train_data= create_data_set(train_data,lookback)
# train_data.shape
train_data = np.reshape(train_data.values,(train_nsamples, lookback, train_data.shape[1]))
validate_data = np.reshape(validate_data.values,(validate_nsamples, lookback, validate_data.shape[1]))
test_data = np.reshape(test_data.values,(test_nsamples, lookback, test_data.shape[1]))

In [ ]:
# test_target.shape
# train_data

In [ ]:
def reshape_target(data, lookback, nsamples):
    unique_sess = data.SessionID.unique()
    y = []
    y=(data.groupby(['SessionID'])).agg(KPI=('Flag', 'max')).reset_index()
#     for id in unique_sess:
#         f=data[data.SessionID == id]['Flag']
#         y = np.append(y,f)
    y['SessionID'] = pd.Categorical(y['SessionID'], 
                                    categories=unique_sess, 
                                    ordered=True)
    y.sort_values('SessionID')
#     print(data.SessionID)
#     print(unique_sess)
#     print(y)
    return y.KPI.to_numpy()

In [ ]:
train_target = reshape_target(train, lookback, train_nsamples)
validate_target = reshape_target(validate, lookback, validate_nsamples)
test_target = reshape_target(test, lookback, test_nsamples)

In [ ]:
test_target

In [ ]:
print(train_target.shape, train_rsd.shape)
# print(validate_target.shape)
# print(test_target.shape)


In [ ]:
###IMPLEMENT DISTANCE BASED CLASS BALANCER HERE
import torch

def dist_class_balancer(dummy_d,dummy_t, k):

    stime=time.time()
    # dummy_d = train_rsd[0:100]
    # dummy_t = train_target[0:100]
#     dummy_d = train_rsd
#     dummy_t = train_target

    larger_sub_idx = np.where(dummy_t == 0)
    smaller_sub_idx = np.where(dummy_t != 0)

    cases_to_keep = smaller_sub_idx

    larger_sub = np.take(dummy_d, larger_sub_idx, axis=0)
    smaller_sub = np.take(dummy_d, smaller_sub_idx, axis=0)

    # np.squeeze(smaller_sub, axis=(2,)).shape

    # print(dummy_d.shape, larger_sub.shape, smaller_sub.shape)

    # print(np.squeeze(smaller_sub, axis=(0,)).shape)

    smaller_torch = torch.flatten(torch.tensor(np.squeeze(smaller_sub, axis=(0,))), start_dim=1)
    larger_torch = torch.flatten(torch.tensor(np.squeeze(larger_sub, axis=(0,))), start_dim=1)


    print(smaller_torch.shape, larger_torch.shape)
    # print(torch.cdist(smaller_torch, larger_torch).topk(10, largest=False))
#     print(len(set(torch.flatten(torch.cdist(smaller_torch, larger_torch).topk(5, largest=False)[1]).detach().numpy())))

    to_keep = np.unique(torch.flatten(torch.cdist(smaller_torch, larger_torch).topk(k, largest=False)[1]).detach().numpy())
    dummy_idx_to_keep = np.concatenate((cases_to_keep[0],to_keep))
    # cases_to_keep.append()

    print(time.time()-stime)
    return( np.unique(dummy_idx_to_keep))
#     return (np.take(dummy_d, dummy_idx_to_keep, axis = 0), np.take(dummy_t, dummy_idx_to_keep, axis = 0))

In [ ]:
train_idx = dist_class_balancer(train_rsd, train_target, 5)

train_ids.sort()
bal_train_ids = np.take(np.squeeze(train_ids), train_idx, axis = 0)
bal_train_rsd = np.take(train_rsd, train_idx, axis = 0)
bal_train_target = np.take(train_target, train_idx, axis = 0)

test_idx = dist_class_balancer(test_rsd, test_target, 5)

test_ids.sort()
bal_test_ids = np.take(np.squeeze(test_ids), test_idx, axis = 0)
bal_test_rsd = np.take(test_rsd, test_idx, axis = 0)
bal_test_target = np.take(test_target, test_idx, axis = 0)

validate_idx = dist_class_balancer(validate_rsd, validate_target, 5)
validate_ids.sort()
bal_validate_ids = np.take(np.squeeze(validate_ids), validate_idx, axis = 0)
bal_validate_rsd = np.take(validate_rsd, validate_idx, axis = 0)
bal_validate_target = np.take(validate_target, validate_idx, axis = 0)

# train_rsd, train_target = dist_class_balancer(train_rsd, train_target, 5)
# test_rsd, test_target = dist_class_balancer(test_rsd, test_target, 5)
# validate_rsd, validate_target = dist_class_balancer(validate_rsd, validate_target, 5)

In [ ]:
train_rsd.shape

In [ ]:
from torch.utils.data import Dataset

class timeseries(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x,dtype=torch.float32)
        self.y = torch.tensor(y,dtype=torch.float32)
        self.len = x.shape[0]

    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
  
    def __len__(self):
        return self.len


    

In [ ]:

def validate_me(x,y):
    test_target = x
#     print(y)
    pred_test_y = (y.detach().numpy() > 0.).astype(int)

    matrix = metrics.confusion_matrix(test_target, pred_test_y ) 
    return("F1:",metrics.f1_score(test_target, pred_test_y ),"Accuracy:",metrics.accuracy_score(test_target, pred_test_y ),"ROC_AUC:",metrics.roc_auc_score(test_target, y.detach().numpy() )) 

In [ ]:
import torch

from torch import nn

number_of_features = train_rsd.shape[2]

hidden_layers = 32

class LSTM_nn(nn.Module):
    def __init__(self):
        super(LSTM_nn,self).__init__()
        self.bn = nn.BatchNorm1d(lookback)
        self.do = nn.Dropout(0.5)
        self.lstm = nn.LSTM(input_size=number_of_features,hidden_size=hidden_layers,num_layers=1,batch_first=True,bidirectional=True,dropout=0.0)
#         self.fc1 = nn.Linear(in_features=hidden_layers,out_features=1)
        self.fc1 = nn.Linear(in_features=hidden_layers*2,out_features=1)

    def forward(self,x):
        output = self.do(x)
        output = self.bn(output)
#         output = self.bn(x)
#         output = output[:,-1,:]
        output,_status = self.lstm(output)
#         print(output.size())
        output = output[:,-1,:]
#         print(output.size())
        output = self.do(output)
        output = self.fc1(torch.relu(output))
#         output = torch.relu(self.fc1(output))
        return output

model = LSTM_nn()

In [ ]:
class Attention(nn.Module):
    def __init__(self, feature_dim, step_dim, bias=True, **kwargs):
        super(Attention, self).__init__(**kwargs)
        
        self.supports_masking = True

        self.bias = bias
        self.feature_dim = feature_dim
        self.step_dim = step_dim
        self.features_dim = 0
        
        weight = torch.zeros(feature_dim, 1, requires_grad=True)
        nn.init.kaiming_uniform_(weight)
        self.weight = nn.Parameter(weight)
        
        if bias:
            self.b = nn.Parameter(torch.zeros(step_dim, requires_grad=True))
        
    def forward(self, x, mask=None):
        feature_dim = self.feature_dim 
        step_dim = self.step_dim

        eij = torch.mm(
            x.contiguous().view(-1, feature_dim), 
            self.weight
        ).view(-1, step_dim)
        
        if self.bias:
            eij = eij + self.b
            
        eij = torch.tanh(eij)
        a = torch.exp(eij)
        
        if mask is not None:
            a = a * mask

        a = a / (torch.sum(a, 1, keepdim=True) + 1e-10)

        weighted_input = x * torch.unsqueeze(a, -1)
        
        return (torch.sum(weighted_input, 1), a)
#         return weighted_input
#         if self.training:
#             return torch.sum(weighted_input, 1)
#         else:
#             return (torch.sum(weighted_input, 1), a)

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn

number_of_features = train_rsd.shape[2]

hidden_layers = 96

class LSTMAttn_nn(nn.Module):
    def __init__(self):
        super(LSTMAttn_nn,self).__init__()
        self.bn = nn.BatchNorm1d(lookback)
        self.do = nn.Dropout(0.5)
        self.lstm = nn.LSTM(input_size=number_of_features,hidden_size=hidden_layers,num_layers=1,batch_first=True,bidirectional=True,dropout=0.0)
#         self.fc1 = nn.Linear(in_features=hidden_layers,out_features=1)
        self.fc1 = nn.Linear(in_features=hidden_layers*2,out_features=1)
        self.attn = nn.Linear(hidden_layers , hidden_layers)
        self.attn_combine = nn.Linear(hidden_layers * 2, hidden_layers)
#         self.attention = Attention(hidden_layers,lookback)
        self.attention = Attention(hidden_layers* 2,lookback )

    def forward(self,x):
        output = self.do(x)
        bn_output = self.bn(output)
#         output = self.bn(x)
#         output = output[:,-1,:]
        lstm_output, _status = self.lstm(bn_output)

        lstm_attn, attn_wts  = self.attention(lstm_output)
        lstm_attn_do = self.do(lstm_attn)
#         relu_output = self.fc1(torch.relu(lstm_attn_do))
        relu_output = torch.relu(self.fc1(lstm_attn_do))
        return(relu_output, attn_wts)
#         if self.training:
#             return relu_output
#         else:
#             return(relu_output, attn_wts)

model = LSTMAttn_nn()

In [ ]:

learning_rate=0.0005

epochs = 100
batch_size=13

# class_weight = torch.tensor([1., 5.])


# criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.MSELoss()
criterion = torch.nn.BCEWithLogitsLoss(reduction='sum')
# optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
optimizer = torch.optim.Adagrad(model.parameters(),lr=learning_rate)
# optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,weight_decay=1e-3)


# dataset = timeseries(train_rsd[0:5000],train_target[0:5000])
dataset = timeseries( bal_train_rsd, bal_train_target)

validate_set = timeseries( bal_validate_rsd, bal_validate_target)
# validate_set = timeseries(validate_rsd[1000:2000],validate_target[1000:2000])

#dataloader

from torch.utils.data import DataLoader 
train_loader = DataLoader(dataset,shuffle=True,batch_size=batch_size)
validate_loader = DataLoader(validate_set,shuffle=True,batch_size=batch_size)

from torch.autograd import Variable

train_total_attn = np.zeros((1,lookback))

for i in range(epochs):
    avg_loss = 0
    train_pred = torch.zeros(len(train_loader))
    train_tar = torch.zeros(len(train_loader))
    for j,data in enumerate(train_loader):
        optimizer.zero_grad()
        model.train()
#         print(data[0][0].view(-1,5,16))
        model_ot = model(data[:][0].view(-1,lookback,number_of_features))
        y_pred = model_ot[0].reshape(-1)
        
        attn = model_ot[1]
#         print(attn.size())
        train_total_attn = np.vstack((train_total_attn,attn.detach().numpy()))
        loss = criterion(y_pred,data[:][1])
        loss.backward()
        clipping_value = 1.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clipping_value)
        optimizer.step()
        
        avg_loss += loss.item() #/ len(train_loader)
        train_pred = torch.cat((train_pred, y_pred), 0)
        train_tar = torch.cat((train_tar, data[:][1]), 0)
        
    if (i+1)%25 == 0 or i==0:
        print(i,"th epoch : ")
#         print(sum((train_tar.detach().numpy()==adj_train_pred))/len(adj_train_pred))
        print("Training metrics: Loss",avg_loss, validate_me(train_tar,train_pred))
        total_attn = np.zeros((1,lookback))
        avg_val_loss = 0
        c=0
        
        val_preds = torch.zeros(len(validate_loader))
        val_targets = torch.zeros(len(validate_loader))
        for k, val in enumerate(validate_loader):
            model.eval()
            model_out = model(val[:][0].view(-1,lookback,number_of_features))
#             print(model_out[0])
            attn = model_out[1]
            total_attn = np.vstack((total_attn,attn.detach().numpy()))
            c+=1
#             print(attn.shape)
            y_epoch = model_out[0].reshape(-1)
            loss_epoch = criterion(y_epoch,val[:][1])
            avg_val_loss += loss_epoch.item() #/ len(validate_loader)
            val_preds = torch.cat((val_preds, y_epoch), 0)
            val_targets = torch.cat((val_targets, val[:][1]), 0)
            
        print("Validation metrics  : Loss",avg_val_loss, validate_me(val_targets,val_preds))
        print("Runs: ",c)
        


In [ ]:
print(len(train_tar))
print(len(train_pred))

In [ ]:
(train_pred>0.).detach().numpy().astype(int)

In [ ]:
(sum((train_tar==train_pred).detach().numpy()))

In [ ]:
# Running predictions on train dataset to obtain attention weights

test_set = timeseries( bal_train_rsd, bal_train_target)
test_loader = DataLoader(test_set,shuffle=False,batch_size=batch_size)


test_total_attn = np.zeros((1,lookback))
avg_test_loss = 0
c=0

test_preds = torch.zeros(len(test_loader))
test_targets = torch.zeros(len(test_loader))
for k, val in enumerate(test_loader):
    model.eval()
    model_out = model(val[:][0].view(-1,lookback,number_of_features))
#             print(model_out[0])
    attn = model_out[1]
    test_total_attn = np.vstack((test_total_attn,attn.detach().numpy()))
    c+=1
#             print(attn.shape)
    y_epoch = model_out[0].reshape(-1)
    loss_epoch = criterion(y_epoch,val[:][1])
    avg_test_loss += loss_epoch.item() #/ len(validate_loader)
    test_preds = torch.cat((test_preds, y_epoch), 0)
    test_targets = torch.cat((test_targets, val[:][1]), 0)
#             val_preds.append(y_epoch)
#             val_targets.append(val[:][1])

# print("Candidate pool metrics  : Loss",avg_val_loss, validate_me(val_targets,val_preds))
# print("Runs: ",c)

In [ ]:
len(test_total_attn)

In [ ]:
total_attn.shape

In [ ]:
attn_df = pd.DataFrame(data=total_attn[1:])    # values
# ...              index=data[1:,0],    # 1st column as index
# ...              columns=data[0,1:])

In [ ]:
# bal_validate_data = validate_data[validate_data['SessionID'].isin(bal_validate_ids)]

validate_pred_attn = pd.DataFrame({'SessionID':bal_validate_ids , 'Predicted_Flag': val_preds.detach().numpy()[:len(bal_validate_ids)]})
print(validate_pred_attn.shape)
validate_pred_attn = pd.concat([validate_pred_attn, attn_df],axis=1)
print(validate_pred_attn.shape)
#, 'Attention_Wt': total_attn[1:] })
# test_clusters = pd.DataFrame({'SessionID': validate_cl.reset_index().SessionID , 'Cluster': val_clusters})

In [ ]:
len(validate_pred_attn.SessionID.unique())

In [ ]:
train_total_attn[len(bal_train_ids)+1:]

In [ ]:
test_attn_df = pd.DataFrame(data=test_total_attn[1:len(bal_train_ids)+1]) 

test_pred_attn = pd.DataFrame({'SessionID':bal_train_ids , 'Predicted_Flag': test_preds.detach().numpy()[:len(bal_train_ids)]})

# test_attn_df = pd.DataFrame(data=train_total_attn[1:len(bal_train_ids)+1]) 

# test_pred_attn = pd.DataFrame({'SessionID':bal_train_ids , 'Predicted_Flag': train_pred.detach().numpy()[:len(bal_train_ids)]})

print(test_pred_attn.shape)



test_pred_attn = pd.concat([test_pred_attn, test_attn_df],axis=1)
print(test_pred_attn.shape)
print(len(test_pred_attn.SessionID.unique()))

In [ ]:
test_pred_attn.describe()
test_pred_attn['Adj_Predicted_Flag'] = (test_pred_attn['Predicted_Flag']>0.).astype(int)

In [ ]:
test_pred_attn

In [ ]:
x=np.array(test_pred_attn['Adj_Predicted_Flag'] )
print(len(x))

In [ ]:
sum(test_pred_attn['Adj_Predicted_Flag']==bal_train_target)

In [ ]:
corr_test_pred_attn = test_pred_attn[test_pred_attn['Adj_Predicted_Flag']==bal_train_target]


In [ ]:
len(corr_test_pred_attn)/len(test_pred_attn)

In [ ]:
len(bal_train_target)

In [ ]:
adj_train_pred=(train_pred>0.).detach().numpy().astype(int)
print(sum((train_tar.detach().numpy()==adj_train_pred))/len(adj_train_pred))

In [ ]:
validate_pred_attn[validate_pred_attn['Predicted_Flag']>1].head(15)

In [ ]:
validate_pred_attn[validate_pred_attn['Predicted_Flag']==0.].head(15)

In [ ]:
validate_pred_attn[validate_pred_attn['SessionID']==10003020]

import time
st = time.time()

a = torch.flatten(torch.tensor(test_rsd[0:10]), start_dim=1)  #simulates running traces
b = torch.flatten(torch.tensor(validate_rsd), start_dim=1) #dataset to find candidates from, also used for tuning the model 

print(torch.cdist(a, b).topk(100, largest=False)[1][0:10])

print( time.time() - st)

def get_topk_ns(running_traces, k):

    a = torch.flatten(torch.tensor(running_traces), start_dim=1)
    b = torch.flatten(torch.tensor(validate_rsd), start_dim=1)
    nn_test = torch.cdist(a, b).topk(k, largest=False)[1]
#     print(torch.cdist(a, b).topk(100)[1][0:10])
    return(nn_test)


In [ ]:
nd = session_wrk_lkbk[['SessionID','TIMESTAMP','PAGE_NAME', 'AgeCategory', 'Gender','Flag']]

np.random.seed(420)


nn_test = nd[nd['SessionID'].isin(np.random.choice(bal_train_ids,10000))]


nn_validate = nd[nd['SessionID'].isin(np.random.choice(bal_validate_ids,250))]


nn_eval = nd[nd['SessionID'].isin(np.random.choice(bal_test_ids,1000))]

In [ ]:
#pre-process eval set - WINDOWSSS
# min_los= lookback/3
min_los = 3
uni_ids = nn_eval.SessionID.unique()
nn_eval_pp = pd.DataFrame()

for id in uni_ids:
    id_sub = nn_eval[nn_eval.SessionID == id]
    los = id_sub.shape[0]
#     print(id_sub.shape)
    for i in np.arange(min_los,los-1,1):
        id_sub.loc[:,'SessionID'] = str(id)+'_'+str(int(i))
#         print(i)
        sub = id_sub.iloc[0:int(i),:]
        nn_eval_pp = nn_eval_pp.append(sub)
        
# nn_eval_pp

In [ ]:
# nn_eval_pp.head(25)

In [ ]:
print(nn_test.shape, nn_test.Flag.sum())

In [ ]:
import time, random

stime= time.time()

running_traces = nn_validate.copy()
# running_traces = nn_validate[nn_validate['Flag']==1]
# min_los = int(lookback/2)
min_los = 3
idx_to_drop = []

case_ids = running_traces.SessionID.unique()

for id in case_ids:
    case_level = running_traces.loc[running_traces['SessionID']==id,:]
    k_max = case_level.shape[0]

    k = random.randint(min_los, k_max)  #random K prefix
#     print(case_level.index.tolist()[k:])
    idx_to_drop.extend(case_level.index.tolist()[k:])
    
running_traces=running_traces.drop(idx_to_drop)

print(time.time()-stime)
print(running_traces.shape, nn_test.shape)

In [ ]:
print(len(case_ids))

In [ ]:
import time, random

stime= time.time()

candidate_pool = nn_test.copy()
# min_los = int(lookback/2)
min_los = 3
idx_to_drop = []

case_ids = candidate_pool.SessionID.unique()

for id in case_ids:
    case_level = candidate_pool.loc[candidate_pool['SessionID']==id,:]
    tlos = case_level.shape[0]

    k = tlos-1
#     print(case_level.index.tolist()[k:])
    idx_to_drop.extend(case_level.index.tolist()[k:])  # n-1 prefix
    
candidate_pool=candidate_pool.drop(idx_to_drop)

print(time.time()-stime)
print(candidate_pool.shape, nn_validate.shape)

In [ ]:
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.util.constants import PARAMETER_CONSTANT_ACTIVITY_KEY

# nd = session_wrk_lkbk[['SessionID','TIMESTAMP','PAGE_NAME']]
# nn_test = nd[nd['SessionID'].isin(test_ids[0:4])]

# # log_csv = dataframe_utils.convert_timestamp_columns_in_df(nn_sub)
# log_csv = nn_sub.sort_values('TIMESTAMP')

# parameters = {log_converter.Variants.TO_EVENT_LOG.value.Parameters.CASE_ID_KEY: 'SessionID'}

# event_log = log_converter.apply(log_csv, parameters=parameters, variant=log_converter.Variants.TO_EVENT_LOG)

# print(event_log)

# from pm4py.algo.conformance.alignments.edit_distance import algorithm as logs_alignments
# parameters = {}
# alignments = logs_alignments.apply(event_log[0], event_log[1], parameters=parameters)

In [ ]:
import pm4py
from pm4py.objects.log.obj import EventLog

def convert_df_to_EventLogObj(df, case_id, activity_key, timestamp_key):
    log = pm4py.format_dataframe(df,
                                 case_id = case_id,
                                 activity_key= activity_key,
                                 timestamp_key= timestamp_key)

    log = pm4py.convert_to_event_log(log)
    
    trace = EventLog()
    for i in range(1,len(log)):
        trace.append(log[i])
    
    return trace


running_traces[running_traces['SessionID']==10169326]

In [ ]:
running_trace_obj = convert_df_to_EventLogObj(running_traces, case_id = "SessionID",
                                     activity_key= "PAGE_NAME",
                                     timestamp_key= "TIMESTAMP")
candidate_pool_obj = convert_df_to_EventLogObj(candidate_pool, case_id = "SessionID",
                                     activity_key= "PAGE_NAME",
                                     timestamp_key= "TIMESTAMP")

eval_set_obj = convert_df_to_EventLogObj(nn_eval_pp, case_id = "SessionID",
                                     activity_key= "PAGE_NAME",
                                     timestamp_key= "TIMESTAMP")

In [ ]:
running_trace_obj[0]

In [ ]:
candidate_pool_obj

In [ ]:
# running_trace_obj
from pm4py.objects.log.util import log_regex

# form a mapping dictionary associating each activity of the two logs to an ASCII character
mapping = log_regex.form_encoding_dictio_from_two_logs(running_trace_obj, candidate_pool_obj, parameters={})
# encode the second log (against which we want to align each trace of the first log)
list_encodings = log_regex.get_encoded_log(candidate_pool_obj, mapping, parameters={})   #ASCII encoding for logs in search space for candidate selection
# # optimization: keep one item per variant
set_encodings = set(list_encodings) # set absrraction
list_encodings = list(set_encodings) #list abstraction

list_encodings = sorted(list_encodings, key=lambda x: len(x)) #sorting the list by length


In [ ]:
cand_pool_seq = {}

for trace in candidate_pool_obj:
    cand_pool_seq[trace._get_attributes()['concept:name']] = log_regex.get_encoded_trace(trace, mapping, parameters={})

In [ ]:
eval_set_seq = {}

for trace in eval_set_obj:
    eval_set_seq[trace._get_attributes()['concept:name']] = log_regex.get_encoded_trace(trace, mapping, parameters={})

In [ ]:
# eval_set_seq

cand_pool_seq

In [ ]:
len(cand_pool_seq)

In [ ]:
len(set_encodings)

In [ ]:
len(list_encodings)

In [ ]:
import stringdist

def get_closest_traces(encoded_trace, list_encodings, dist_thresh):
#     dist_thresh = 0.4
    closest_trace = list()
    c=-1
    for i in range(0, len(list_encodings)):
    #     print(len(list_encodings[i]))
        dist = (stringdist.levenshtein_norm( encoded_trace, list_encodings[i]))
    #     dist = (stringdist.levenshtein( encoded_trace, list_encodings[i]))
    #     if c == -1:
    #         min_dist=dist
        if dist <= dist_thresh:
            min_dist=dist
            closest_trace.append(list_encodings[i])
    #         closest_trace.append(log_regex.get_encoded_log(list_encodings[i], mapping, parameters={}))
            c += 1
#         elif dist == min_dist:
#             closest_trace.append(list_encodings[i])
#             c += 1


#     print(min_dist,c)
    return(closest_trace)

In [ ]:
validate_pred_attn.tail()

In [ ]:
(session_wrk_lkbk[session_wrk_lkbk['SessionID']==55292373]['PAGE_NAME'])

In [ ]:
# validate_pred_attn.iloc[:,2:].sum(axis=1)
validate_pred_attn.loc[validate_pred_attn['SessionID']==55292373,:].iloc[:,2:]

In [ ]:

from weighted_levenshtein import lev, osa, dam_lev

def get_attn_wts(rt, idc, cand_id):
#     print(idc)
    runn_attn_wts = validate_pred_attn.loc[validate_pred_attn['SessionID']==(int(idc)),:].iloc[:,2:]
    cand_attn_wts = test_pred_attn.loc[test_pred_attn['SessionID']==(int(cand_id)),:].iloc[:,2:]
#     print(attn_wts)
    f = validate_pred_attn.loc[validate_pred_attn['SessionID']==(int(idc)),'Predicted_Flag']
    # print(f.any())
    pred_flag = int(f.any())
    # print(pred_flag)

    sign_mul = 1 if pred_flag==0 else -1
#     print(sign_mul)
    n=9
#     print(cand_pool_seq[str(idc)])
    los = len(cand_pool_seq[str(cand_id)])

    substitute_costs = np.ones((128, 128), dtype=np.float64)

    for i in np.arange(los,0,-1):
        
        curr_acc = cand_pool_seq[str(cand_id)][i-1]
#         print(len(cand_attn_wts.loc[:,:]), i)
        attwt = (cand_attn_wts.iloc[:,i-1])
        if attwt.any() > float(1/lookback):
            curr_acc_cost = (attwt) - 1
    #         print(curr_acc_cost)
            substitute_costs[ord(curr_acc), :] = curr_acc_cost
    
    los = len(rt)
    
    for i in np.arange(los,0,-1):
        
        curr_acc = rt[i-1]
        attwt = (runn_attn_wts.iloc[:,i-1])
        if attwt.any() > float(1/lookback):
            curr_acc_cost = (attwt ) + 1
    #         print(curr_acc_cost)
            substitute_costs[:, ord(curr_acc)] = curr_acc_cost

    return substitute_costs
#     print(curr_acc_cost)


In [ ]:
import stringdist

def get_closest_traces_attn(encoded_trace, trace_id, dist_thresh):
#     dist_thresh = 0.4
    closest_trace = list()
    c=-1
#     sub_costs = get_attn_wts(trace_id)
    word = encoded_trace
    for i in cand_pool_seq:
#         print(i)
        sub_costs = get_attn_wts(word, trace_id, i)
        word1 = cand_pool_seq[i]
        dist = lev(word,word1, substitute_costs=sub_costs)/max(len(word),len(word1))
    #     print(len(list_encodings[i]))
#         dist = (stringdist.levenshtein_norm( encoded_trace, list_encodings[i]))
        if dist <= dist_thresh:
            min_dist=dist
            closest_trace.append(i)
            c += 1
    return(closest_trace)

In [ ]:
def get_eval_traces(encoded_trace, trace_id, dist_thresh):
#     dist_thresh = 0.4
    closest_trace = list()
    c=-1
#     sub_costs = get_attn_wts(trace_id)
    word = encoded_trace
    for i in eval_set_seq:
#         print(i)
#         sub_costs = get_attn_wts(word, trace_id, i)
        word1 = eval_set_seq[i]
        dist = lev(word,word1)/max(len(word),len(word1))
    #     print(len(list_encodings[i]))
#         dist = (stringdist.levenshtein_norm( encoded_trace, list_encodings[i]))
        if dist <= dist_thresh:
            min_dist=dist
            closest_trace.append(i)
            c += 1
    return(closest_trace)

In [ ]:
sim_dist_thresh = 0.5
eval_dist_thresh = 0.5

In [ ]:
st=time.time()

num_matches = np.zeros(len(running_trace_obj))
num_seqmatches = np.zeros(len(running_trace_obj))
set_of_close_trace = {}
e=0
i=0
ids_for_close_trace = {}

ids_for_Eval_trace = {}

for trace in running_trace_obj:
#     print(trace)
    encoded_trace = log_regex.get_encoded_trace(trace, mapping, parameters={}) #ASCII encoding for running trace
    trace_id = trace._get_attributes()['concept:name']
    
    closest_trace_ids = get_closest_traces_attn(encoded_trace, trace_id, sim_dist_thresh)
    
    eval_trace_ids = get_eval_traces(encoded_trace, trace_id, eval_dist_thresh)
    ids_for_Eval_trace[trace_id] = eval_trace_ids
    if len(closest_trace_ids) == 0:
        e+=1
#         closest_traces = get_closest_traces(encoded_trace, list_encodings, 0.4)
    ids_for_close_trace[trace_id] = closest_trace_ids

    num_matches[i] = len(closest_trace_ids) 
    i+=1
    
print(time.time()-st)

print(e)

In [ ]:
from pm4py.util import string_distance
import difflib
from pm4py.objects.log.util import log_regex

num_matches = np.zeros(len(running_trace_obj))
num_seqmatches = np.zeros(len(running_trace_obj))
set_of_close_trace = {}
e=0
i=0
for trace in running_trace_obj:
#     print(trace)
    encoded_trace = log_regex.get_encoded_trace(trace, mapping, parameters={}) #ASCII encoding for running trace
    closest_traces = get_closest_traces(encoded_trace, list_encodings, sim_dist_thresh)
    if len(closest_traces) == 0:
        e+=1
#         closest_traces = get_closest_traces(encoded_trace, list_encodings, 0.4)
    set_of_close_trace[trace._get_attributes()['concept:name']] = closest_traces
#     if encoded_trace in set_encodings:  #if exact match is found
#         argmin_dist = encoded_trace
# #         print("Found")
#     else:
#         argmin_dist = string_distance.argmin_levenshtein(encoded_trace, list_encodings)  #returns closest string
    #     print(argmin_dist)
    c=0
#     for tr in closest_traces:
#         seq_match = difflib.SequenceMatcher(None, encoded_trace, tr).get_matching_blocks()  #finds matching sequences in closest string
#         c += (len(seq_match) - 1)
#     num_seqmatches[i] = c
    num_matches[i] = len(closest_traces) 
    i+=1
    
print(e)

In [ ]:
st = time.time()

ids_for_close_trace0 = {}
for k in set_of_close_trace:
    ids_for_close_trace0[k]=[]
    for seq in set_of_close_trace[k]:
        for k2 in cand_pool_seq:
            if seq==cand_pool_seq[k2]:
                ids_for_close_trace0[k].append(k2.rstrip())
                
print(time.time()-st)

In [ ]:
def pop_prefix(prefix, los, n):
    if len(prefix.split('-')) <= n:
        return prefix
    else:
        split_prefix = prefix.split('-')[1:]
        pref='-'.join(str(e) for e in split_prefix)
        return pref
            
def update_los(pref,los,n):
    if len(pref.split('-'))>n:
        return los
    else:
        return los+1

def create_transys_n(sess_page_level,n):
    transys = {}
    unique_sess =  sess_page_level['SessionID'].unique()
    sess_feat_row = 0
    los = 0
    for sess in unique_sess:
#         sess_feat.at[sess_feat_row,'SessionID'] = sess
        session_level = sess_page_level.loc[sess_page_level['SessionID']==sess]
#         print(session_level.shape[0])
        first_page=1
        c = 0
        prefix = "SOS"
        prev=""
        for index,row in session_level.iterrows():
            c +=1
            
            prefix = pop_prefix(prefix, los, n)
#             print(prefix)
            curr = session_level.at[index,'PAGE_NAME']
            if first_page==1:
                first_page=0
#                 prefix.append(curr)
#             if prev!=curr:
                prefix = prefix + '-' + curr
#                 print("1", prev, curr, prefix)
                los = 1
#                     los += 1
                prev=curr
#                 print(prefix)
                
#                 continue
            else:
#                 print(prev,curr)
                if prev!=curr:
                    if prefix in transys.keys() and curr not in transys.get(prefix):
                        
                        transys.setdefault(prefix, []).append(curr)
#                         print(str(transys))
#                 prefix.append(curr)
                        prefix = prefix + '-' + curr
#                         print("2", prev, curr, prefix)
#                         los = update_los(prefix,los,n)
                        los += 1
                    elif prefix not in transys.keys():
#                         prefix = pop_prefix(prefix, los, n)
                        transys.setdefault(prefix, []).append(curr)
                        prefix = prefix + '-' + curr
#                         print("3", prev, curr, prefix)
#                         los = update_los(prefix,los,n)
                        los += 1
#                         print(str(transys))
                    prev=curr
#                     print("Adding")
                else:
                    if prefix not in transys.keys():
                        transys.setdefault(prefix, []).append(curr)
                    elif curr not in transys.get(prefix):
                        transys.setdefault(prefix, []).append(curr)
#             if c==session_level.shape[0]:
#                 prefix = pop_prefix(prefix, los, n)
#                 if prefix in transys.keys() and "EOS" not in transys.get(prefix):
#                     transys.setdefault(prefix, []).append("EOS")
#                 elif prefix not in transys.keys():
#                     transys.setdefault(prefix, []).append("EOS")
#                     print("Skipping")
    
#     bigram_df = pd.DataFrame(vect.fit_transform(sess_feat['twograms'].values).todense(), columns = vect.get_feature_names())

#     final_df = pd.concat([sess_feat,bigram_df], axis=1)
    return(transys)

In [ ]:
# ids_for_close_trace

l1 = set(ids_for_close_trace['10668376'])

In [ ]:
# ids_for_Eval_trace

In [ ]:
# !pip install statsmodels

In [ ]:
from scipy import stats
from statsmodels.stats import weightstats as stests

c=0
e=0
npk = 0 #No key in positive outcomes subset
nnk = 0 #No key in negative outcomes subset
nra = 0 #No reccomended activities
knf = 0 #Running trace last state not found in eval TS
nevtr = 0 #No evaluation traces ffound

tot_cnt=0
pos_trc_cnt=0
pos_rec_cnt=0
neg_rec_cnt=0

valid_rec_cnt=0

ev_trc_cnt=0
pos_sub_cnt=0
neg_sub_cnt=0

pos_sub_imp=0
pos_sub_wrs=0
pos_sub_imp_sg=0
pos_sub_wrs_sg=0

results = {}

pot_recs_for_running_tr = {}

id_for_recofoll = {}
id_for_reconfoll = {}


st = time.time()

for id in ids_for_close_trace:
    tot_cnt+=1
    l1 = set(ids_for_close_trace[str(id)])
    l2 = set(ids_for_close_trace0[str(id)])
    if(len(l1)==0):
        e+=1
    if(l1==l2):
#         print(l1, l2)
        c+=1
#     print(id)
    if(len(l1)!=0):
        pos_trc_cnt+=1
        
    #     print(running_traces[running_traces['SessionID']==id])
        #get state of running trace
    #     print(create_transys_n(running_traces[running_traces['SessionID']==int(id)].sort_values(['TIMESTAMP']),2).keys())
        rt_state = list(create_transys_n(running_traces[running_traces['SessionID']==int(id)].sort_values(['TIMESTAMP']),2).keys())[-1]
        #get transition system from similar traces
#         print(list(l1))
        closest_df = nn_test[nn_test['SessionID'].isin(list(map(int,l1)))]
#         print(closest_df)
        closest_df_pos = closest_df[closest_df['Flag']==0]
        closest_df_neg = closest_df[closest_df['Flag']==1]

        ts_closest_df_pos= create_transys_n(closest_df_pos.sort_values(['SessionID','TIMESTAMP']),2)
        ts_closest_df_neg= create_transys_n(closest_df_neg.sort_values(['SessionID','TIMESTAMP']),2)
        
#         pos_rec
#         neg_rec

#         print(rt_state)
        reco = []
        pos_rec=[]
        neg_rec=[]
    
#         print(ts_closest_df_pos)
#         print(ts_closest_df_neg)
        f0 = rt_state not in list(ts_closest_df_pos.keys())
        f1 =  rt_state not in list(ts_closest_df_neg.keys())
        f2 = 0
        if f0:
            npk += 1
#             print("No pos pot recos")
#             reco = []
        else:
            pos_rec_cnt+=1
            pos_rec = ts_closest_df_pos[rt_state]
#             print("Pos out potential recos :", pos_rec)
        if f1:
            nnk += 1
#             print("No neg pot recos")
            reco=pos_rec
            f2=1
        else:
            neg_rec_cnt+=1
            neg_rec = ts_closest_df_neg[rt_state]
#             print("Neg out potential recos :", neg_rec)
#         print(ts_closest_df_pos[rt_state], ts_closest_df_neg[rt_state])
        
#         if(len(neg_rec)==0):
#                 print("No neg pot recos")
#                 reco=pos_rec
#         else:
        if not f0 and not f1 and not f2:
            for acc in pos_rec:
                if acc not in neg_rec:
                    reco.append(acc)
                    
        if len(reco)>0:
            valid_rec_cnt+=1
            pot_recs_for_running_tr[id]=reco
#             print("Recommended activity", reco)
            
            pos_reco_foll=0
            pos_reco_nfoll=0
            
            
            
            tot = len(ids_for_Eval_trace[id])
            
            if tot>0:
                ev_trc_cnt+=1
#                 print(tot)
                eval_trace_ids = ids_for_Eval_trace[id]
                foll_ids=[]
                nfoll_ids=[]
                for eval_id in eval_trace_ids:
#                     print(id, eval_id)
                    df = nn_eval_pp.loc[nn_eval_pp['SessionID']==(eval_id),['SessionID','TIMESTAMP','PAGE_NAME']]
#                     print(df.shape)
                    eval_trace_ts = create_transys_n(df.sort_values(['SessionID','TIMESTAMP']),2)
                    if rt_state in eval_trace_ts.keys():
                        next_eve = eval_trace_ts[rt_state]
#                         print(next_eve, reco)
                        if len(set(next_eve).intersection(set(reco)))>0:
                            pos_reco_foll+=1
                            foll_ids.append(eval_id)
                        else:
                            pos_reco_nfoll+=1
                            nfoll_ids.append(eval_id)
                    
                    else: 
                        nfoll_ids.append(eval_id)
                        pos_reco_nfoll+=1
                        knf+=1
#                 tot = pos_reco_foll+pos_reco_nfoll
#                 if tot>0:
                id_for_recofoll[id]= foll_ids
                id_for_reconfoll[id]=  nfoll_ids  
#                 print("Eval results (Total -- Recco foll -- Recco not followed):", tot, pos_reco_foll/tot, pos_reco_nfoll/tot)
                results[id]=(tot, pos_reco_foll/tot, pos_reco_nfoll/tot)
                if pos_reco_foll!=0:
                    pos_sub_cnt+=1
                    a = nn_eval_pp[nn_eval_pp['SessionID'].isin(foll_ids)]['Flag']
#                     print(sum(nn_eval_pp[nn_eval_pp['SessionID'].isin(foll_ids)]['Flag'])/len((nn_eval_pp[nn_eval_pp['SessionID'].isin(foll_ids)]['Flag'])))
                if pos_reco_nfoll!=0:
                    neg_sub_cnt+=1
                    b = nn_eval_pp[nn_eval_pp['SessionID'].isin(nfoll_ids)]['Flag']
#                     print(sum(nn_eval_pp[nn_eval_pp['SessionID'].isin(nfoll_ids)]['Flag'])/len((nn_eval_pp[nn_eval_pp['SessionID'].isin(nfoll_ids)]['Flag'])))
                if pos_reco_foll!=0 and pos_reco_nfoll!=0:
                    
                    ztest ,pval1 = stests.ztest(a, x2=b, value=0,alternative='two-sided')
#                     print("P-value: ",float(pval1))
                    if(np.mean(a)/np.mean(b)>1):
                        pos_sub_imp+=1
                        if(pval1 < 0.05):
                            pos_sub_imp_sg+=1
                    else:
                        pos_sub_wrs+=1
                        if(pval1 < 0.05):
                            pos_sub_wrs_sg+=1
                        
#                         print("Stat sign.")
            else:
                nevtr+=1
#             eval_set_seq
        else:
            nra+=1
#             print("No reco!")
            
print(time.time()-st)
    
print(e, c, npk, nnk, nra)
print("Total running traces checked:", tot_cnt)
print("Similar traces found for :", pos_trc_cnt)
print("Pos outcome recco found for:", pos_rec_cnt)
print("Neg outcome recco found for:", neg_rec_cnt)

print("Valid recco found for:",valid_rec_cnt)

print("Eval traces found for", ev_trc_cnt)
print("Pos sub found for", pos_sub_cnt)
print("Neg sub found for", neg_sub_cnt)
print("Pos sub outcome improvement for", pos_sub_imp)
print("Stat sig imp for", pos_sub_imp_sg)
print("Pos sub outcome worsens for", pos_sub_wrs)
print("Stat sig wrse for", pos_sub_wrs_sg)


In [ ]:
# results

In [ ]:
tot_foll = 0
tot_nfoll = 0
for g in id_for_recofoll:
    tot_foll+=len(id_for_recofoll[g])
    
for g in id_for_reconfoll:
    tot_nfoll+=len(id_for_reconfoll[g])

In [ ]:
# id_for_recofoll

In [ ]:
from scipy import stats
from statsmodels.stats import weightstats as stests

c=0
e=0
npk = 0 #No key in positive outcomes subset
nnk = 0 #No key in negative outcomes subset
nra = 0 #No reccomended activities
knf = 0 #Running trace last state not found in eval TS
nevtr = 0 #No evaluation traces ffound

tot_cnt=0
pos_trc_cnt=0
pos_rec_cnt=0
neg_rec_cnt=0

valid_rec_cnt=0

ev_trc_cnt=0
pos_sub_cnt=0
neg_sub_cnt=0

pos_sub_imp=0
pos_sub_wrs=0
pos_sub_imp_sg=0
pos_sub_wrs_sg=0

results = {}

pot_recs_for_running_tr = {}

id_for_recofoll = {}
id_for_reconfoll = {}


st = time.time()

for id in ids_for_close_trace:
    tot_cnt+=1
    l1 = set(ids_for_close_trace[str(id)])
    l2 = set(ids_for_close_trace0[str(id)])
    if(len(l2)==0):
        e+=1
    if(l1==l2):
#         print(l1, l2)
        c+=1
#     print(id)
    if(len(l2)!=0):
        pos_trc_cnt+=1
        
    #     print(running_traces[running_traces['SessionID']==id])
        #get state of running trace
    #     print(create_transys_n(running_traces[running_traces['SessionID']==int(id)].sort_values(['TIMESTAMP']),2).keys())
        rt_state = list(create_transys_n(running_traces[running_traces['SessionID']==int(id)].sort_values(['TIMESTAMP']),2).keys())[-1]
        #get transition system from similar traces
#         print(list(l1))
        closest_df = nn_test[nn_test['SessionID'].isin(list(map(int,l2)))]
#         print(closest_df)
        closest_df_pos = closest_df[closest_df['Flag']==0]
        closest_df_neg = closest_df[closest_df['Flag']==1]

        ts_closest_df_pos= create_transys_n(closest_df_pos.sort_values(['SessionID','TIMESTAMP']),2)
        ts_closest_df_neg= create_transys_n(closest_df_neg.sort_values(['SessionID','TIMESTAMP']),2)
        
#         pos_rec
#         neg_rec

#         print(rt_state)
        reco = []
        pos_rec=[]
        neg_rec=[]
    
#         print(ts_closest_df_pos)
#         print(ts_closest_df_neg)
        f0 = rt_state not in list(ts_closest_df_pos.keys())
        f1 =  rt_state not in list(ts_closest_df_neg.keys())
        f2 = 0
        if f0:
            npk += 1
#             print("No pos pot recos")
#             reco = []
        else:
            pos_rec_cnt+=1
            pos_rec = ts_closest_df_pos[rt_state]
#             print("Pos out potential recos :", pos_rec)
        if f1:
            nnk += 1
#             print("No neg pot recos")
            reco=pos_rec
            f2=1
        else:
            neg_rec_cnt+=1
            neg_rec = ts_closest_df_neg[rt_state]
#             print("Neg out potential recos :", neg_rec)
#         print(ts_closest_df_pos[rt_state], ts_closest_df_neg[rt_state])
        
#         if(len(neg_rec)==0):
#                 print("No neg pot recos")
#                 reco=pos_rec
#         else:
        if not f0 and not f1 and not f2:
            for acc in pos_rec:
                if acc not in neg_rec:
                    reco.append(acc)
                    
        if len(reco)>0:
            valid_rec_cnt+=1
            pot_recs_for_running_tr[id]=reco
#             print("Recommended activity", reco)
            
            pos_reco_foll=0
            pos_reco_nfoll=0
            
            
            
            tot = len(ids_for_Eval_trace[id])
            
            if tot>0:
                ev_trc_cnt+=1
#                 print(tot)
                eval_trace_ids = ids_for_Eval_trace[id]
                foll_ids=[]
                nfoll_ids=[]
                for eval_id in eval_trace_ids:
#                     print(id, eval_id)
                    df = nn_eval_pp.loc[nn_eval_pp['SessionID']==(eval_id),['SessionID','TIMESTAMP','PAGE_NAME']]
#                     print(df.shape)
                    eval_trace_ts = create_transys_n(df.sort_values(['SessionID','TIMESTAMP']),2)
                    if rt_state in eval_trace_ts.keys():
                        next_eve = eval_trace_ts[rt_state]
#                         print(next_eve, reco)
                        if len(set(next_eve).intersection(set(reco)))>0:
                            pos_reco_foll+=1
                            foll_ids.append(eval_id)
                        else:
                            pos_reco_nfoll+=1
                            nfoll_ids.append(eval_id)
                    
                    else: 
                        nfoll_ids.append(eval_id)
                        pos_reco_nfoll+=1
                        knf+=1
#                 tot = pos_reco_foll+pos_reco_nfoll
#                 if tot>0:
                id_for_recofoll[id]= foll_ids
                id_for_reconfoll[id]=  nfoll_ids  
#                 print("Eval results (Total -- Recco foll -- Recco not followed):", tot, pos_reco_foll/tot, pos_reco_nfoll/tot)
                results[id]=(tot, pos_reco_foll/tot, pos_reco_nfoll/tot)
                if pos_reco_foll!=0:
                    pos_sub_cnt+=1
                    a = nn_eval_pp[nn_eval_pp['SessionID'].isin(foll_ids)]['Flag']
#                     print(sum(nn_eval_pp[nn_eval_pp['SessionID'].isin(foll_ids)]['Flag'])/len((nn_eval_pp[nn_eval_pp['SessionID'].isin(foll_ids)]['Flag'])))
                if pos_reco_nfoll!=0:
                    neg_sub_cnt+=1
                    b = nn_eval_pp[nn_eval_pp['SessionID'].isin(nfoll_ids)]['Flag']
#                     print(sum(nn_eval_pp[nn_eval_pp['SessionID'].isin(nfoll_ids)]['Flag'])/len((nn_eval_pp[nn_eval_pp['SessionID'].isin(nfoll_ids)]['Flag'])))
                if pos_reco_foll!=0 and pos_reco_nfoll!=0:
                    
                    ztest ,pval1 = stests.ztest(a, x2=b, value=0,alternative='two-sided')
#                     print("P-value: ",float(pval1))
                    if(np.mean(a)/np.mean(b)>1):
                        pos_sub_imp+=1
                        if(pval1 < 0.05):
                            pos_sub_imp_sg+=1
                    else:
                        pos_sub_wrs+=1
                        if(pval1 < 0.05):
                            pos_sub_wrs_sg+=1
                        
#                         print("Stat sign.")
            else:
                nevtr+=1
#             eval_set_seq
        else:
            nra+=1
#             print("No reco!")
            
print(time.time()-st)
    
print(e, c, npk, nnk, nra)
print("Total running traces checked:", tot_cnt)
print("Similar traces found for :", pos_trc_cnt)
print("Pos outcome recco found for:", pos_rec_cnt)
print("Neg outcome recco found for:", neg_rec_cnt)

print("Valid recco found for:",valid_rec_cnt)

print("Eval traces found for", ev_trc_cnt)
print("Pos sub found for", pos_sub_cnt)
print("Neg sub found for", neg_sub_cnt)
print("Pos sub outcome improvement for", pos_sub_imp)
print("Stat sig imp for", pos_sub_imp_sg)
print("Pos sub outcome worsens for", pos_sub_wrs)
print("Stat sig wrse for", pos_sub_wrs_sg)
